In [88]:
import os
import subprocess
import Bio
from Bio import SeqIO 
from Bio import AlignIO
from Bio.Align import MultipleSeqAlignment
import glob
from Bio import SeqRecord
import pandas as pd
import genomicranges as gr
from Bio.Seq import Seq
import pyranges as pr
from Bio import AlignIO, SeqIO

In [2]:
def make_sbatch_file(filename):
    
    my_list = ["#!/bin/bash", 
               "#SBATCH --job-name=all_bz", 
               "#SBATCH --nodes=1",  
               "#SBATCH --ntasks=1",                     
               "#SBATCH --cpus-per-task=10",              
               "#SBATCH --mem=20gb",                    
               "#SBATCH --partition=20",                
               "##SBATCH --output all_bz-%j.out",  
               "#SBATCH --mail-type=ALL",               
               "#SBATCH --mail-user=hlharris@wi.mit.edu"] 
    
    with open(filename, "w") as file: 
        for item in my_list:
            file.write(item + '\n') 
    
def calc_zeros(alignment): 
    count = 0 
    return [0 if base == "-" else (count := count + 1) for base in alignment[0]]


def crop_alignment(start, end, alignment):
        
    zeros_seq = calc_zeros(alignment)
    #print(zeros_seq) #MIGHT NEED TO SUBTRACT 1 FROM EACH 
    try:
        ix_start = zeros_seq.index(start)
        #print(ix_start)
        ix_end = zeros_seq.index(end) #pretty elegant
        #print(ix_end)
        cropped_alignment = alignment[:, ix_start:ix_end]
        return cropped_alignment
    except: 
        return None
    
    
def concat_alignment(gtf_file, alignment):
   # print(alignment)
    complete_align_type = MultipleSeqAlignment([]) 
    #add the groups to complete align: 
    for recordix in range(len(alignment)): 
        #new_record = SeqRecord.SeqRecord("") 
        new_record = SeqRecord("")
        new_record.id = alignment[recordix].id 
        new_record.seq = Seq("") #added this 

        complete_align_type.append(new_record)
    
   # print(complete_align_type)
    for index, row in gtf_file.iterrows():     
        crop_align = crop_alignment(row[3], row[4], alignment)
        #print(crop_align)
        if crop_align is not None: 
            for recordix in range(len(crop_align)): 
                complete_align_type[recordix].seq += crop_align[recordix].seq #append additional sequence
        
    return complete_align_type

In [3]:
#make directory for all the primate sequences 

genes = ["EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 

for gene in genes: 
    os.chdir(myPath + "/sequences/primates/" ) 
    subprocess.run(['mkdir', gene],  stderr=subprocess.PIPE) 
    


NameError: name 'os' is not defined

In [13]:
#saving dictionary for pileated gibbon 
dicty = SeqIO.to_dict(SeqIO.parse('/lab/page_scratch/hannah/pileatedgibbon/' + "genomic", format = "fasta")) 

fasta_file = dicty['LG04']     
SeqIO.write(fasta_file, '/lab/page_scratch/hannah/pileatedgibbon/LGO4.fa', 'fasta') 

fasta_file = dicty['chrY_decollapsed']     
SeqIO.write(fasta_file, '/lab/page_scratch/hannah/pileatedgibbon/chrY.fa', 'fasta') 

fasta_file = dicty['chrX']     
SeqIO.write(fasta_file, '/lab/page_scratch/hannah/pileatedgibbon/chrX.fa', 'fasta')

1

In [ ]:
#once the files are downloaded for each animal/chromosome 

x_genes = ["EIF1AX",  "KDM5C", "KDM6A", "ZFX",  "DDX3X",  "USP9X",  "RPS4X"] 
genes = ["EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 
species = ["gorilla",  "orangutan", "pileatedgibbon"]  


seq_df = pd.read_csv(myPath + '/tables/sequence_gorilla_orang.csv', delimiter="\t" ) #to get coordinates, blasted human sequence to these genomes

for animal in species: 
    
    directory = "/lab/page_scratch/hannah/" + animal + "/" #replace location where genomes are 
    for gene in genes: 

        seq_df1 = seq_df[seq_df['gene_name'] == gene] 
        seq_df2 = seq_df1[seq_df1['animalz'] == animal]
       
        start = int(seq_df2['start']) 
        
        end =   int(seq_df2['end']) 
        
        if gene in x_genes:
            fasta_file = SeqIO.read(directory + "chrX.fa", "fasta") 
        else: 
            fasta_file = SeqIO.read(directory + "chrY.fa", "fasta")
        
        fasta_seq = fasta_file.seq
        
        os.chdir(myPath + '/sequences/primates/' + gene)
               
        sliced_sequence = fasta_seq[start:end]
        records = [SeqIO.SeqRecord(sliced_sequence, id=animal)] 
        SeqIO.write(records, animal, 'fasta') 
        

In [74]:
#take the region alignments, and remove any region with gaps_rename *filtered*

In [ ]:
#repeat mask all files for each species masking primate repeats (hard mask) 
#resubmit for alignment - rename everything with 'masked' 

genes = ["EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 
species = ["human"] #, "chimp", "mac", "marm","gorilla",  "orangutan", "pileatedgibbon", "lemur", "loris"]  

os.chdir(myPath + '/sequences/primates/')

with open("repeat_mask_commands1.py", "a") as file: 
    file.write('import os' + ' \n')
    file.write('import subprocess' + ' \n') 

    for gene in genes:
     
        file.write('os.chdir(' + myPath + '"/sequences/primates/' + gene + '")'  + ' \n')
        
        for specy in species:
            
            file.write('subprocess.run(["RepeatMasker", "-noint", "-species", "primates", "' + specy + '"''],  stderr=subprocess.PIPE)' + ' \n')
            
            
make_sbatch_file("run_masker1.sh") 

with open("run_masker1.sh", "a") as file: 
    print('here')
    file.write('python repeat_mask_commands1.py') 



## Run alignment 

In [ ]:
#after masking step is done -- repeat alignments with everything labelled 'masked'

genes = ["EIF1AX", "EIF1AY", "KDM5D" , "KDM5C","UTY", "KDM6A", "ZFY", "ZFX", "DDX3Y" ,"DDX3X", "USP9Y" , "USP9X", "RPS4Y1", "RPS4X"] 

species_dict = {0:"humanmasked", 1:"chimp", 2: "mac", 3: "marm"} 

files_generated = [] 

for gene in genes:

    records = list(SeqIO.parse(myPath + '/sequences/' + gene + ".fa", "fasta"))
    
    
    #set the trees for input 
    if len(records) == 3: 
        tree = '((((((humanmasked chimp) gorilla) orangutan) pileatedgibbon) mac) lemur loris)'
    else: 
        tree = '((((((humanmasked chimp) gorilla) orangutan) pileatedgibbon) mac marm) lemur loris)'
    
    #make a new directory for each gene: 
    os.chdir(myPath + '/sequences/primates/'+ gene)

    
    subprocess.run(["cp", "human.masked", "humanmasked"])

    subprocess.run(["touch", "all_bz_masked.log"])  #making a new file 
    
    with open("all_bz_masked.log", "w") as log_file:
        result = subprocess.run(['all_bz', '-', tree], stdout=log_file, stderr=subprocess.PIPE, check=True)
        
    make_sbatch_file("testj1_masked.sh") #make sbatch file - it makes a new one each time through so you dont have to delete the old one 
    print(gene) 
    with open("testj1_masked.sh", "a") as file: #append to the file 
        file.write("bash ") 
        file.write("all_bz_masked.log" + "\n")
        file.write("tba '" + tree + "' *.*.maf tba.maf >&tba.log" + "\n") 
        file.write("maf_project tba.maf humanmasked '" + tree + "' > humanmasked_proj.maf" + "\n") 
        file.write("msa_view -o FASTA humanmasked_proj.maf > " + gene + "_msamasked.fa")
        
    subprocess.run(["sbatch", 'testj1_masked.sh'], stderr=subprocess.PIPE)
    
    files_generated = [] 
    os.chdir(myPath + '/sequences/primates/')  